## 1. Environment Setup

In [ ]:
# Clone repository
!git clone https://github.com/YOUR_REPO/HiERO_for_egovlp.git
%cd HiERO_for_egovlp

In [ ]:
# Install dependencies
!pip install -r requirements.txt -f https://data.pyg.org/whl/torch-2.4.0+cu124.html --extra-index-url https://download.pytorch.org/whl/

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

## 2. Path Configuration

In [ ]:
import os

# ================= MODIFY THESE PATHS =================
PROJECT_ROOT = "/content/HiERO_for_egovlp"
DRIVE_ROOT = "/content/drive/MyDrive/AMLproject"

# EgoVLP features (from your screenshot)
EGOVLP_FEATURES_DIR = os.path.join(DRIVE_ROOT, "Captain_Cook_dataset/features/segments/egovlp")

# Annotations
ANNOTATIONS_PATH = os.path.join(PROJECT_ROOT, "annotations/annotation_json/complete_step_annotations.json")
SPLIT_FILE = os.path.join(PROJECT_ROOT, "er_annotations/recordings_combined_splits.json")

# Output directory (save to Drive for persistence)
OUTPUT_DIR = os.path.join(DRIVE_ROOT, "substep1_outputs")
os.makedirs(OUTPUT_DIR, exist_ok=True)

print("=== Paths ===")
print(f"EgoVLP features: {EGOVLP_FEATURES_DIR}")
print(f"Annotations: {ANNOTATIONS_PATH}")
print(f"Output: {OUTPUT_DIR}")

In [ ]:
# Verify paths
import os

print("=== Verifying Paths ===")
if os.path.exists(EGOVLP_FEATURES_DIR):
    npz_files = [f for f in os.listdir(EGOVLP_FEATURES_DIR) if f.endswith('.npz')]
    print(f"✅ EgoVLP features: {len(npz_files)} files")
else:
    print(f"❌ Features not found: {EGOVLP_FEATURES_DIR}")

if os.path.exists(ANNOTATIONS_PATH):
    print(f"✅ Annotations found")
else:
    print(f"❌ Annotations not found: {ANNOTATIONS_PATH}")

## 3. Load Annotations & Features

In [ ]:
import json
import numpy as np

# Load annotations
with open(ANNOTATIONS_PATH, 'r') as f:
    annotations = json.load(f)

print(f"Total annotated videos: {len(annotations)}")

# Sample one annotation
sample_id = list(annotations.keys())[0]
sample = annotations[sample_id]
print(f"\nSample video: {sample_id}")
print(f"Keys: {sample.keys()}")
if 'steps' in sample:
    print(f"Number of steps: {len(sample['steps'])}")

In [ ]:
# Load split information (train/val/test)
with open(SPLIT_FILE, 'r') as f:
    splits = json.load(f)

print(f"Train: {len(splits.get('train', []))} videos")
print(f"Val: {len(splits.get('val', []))} videos")
print(f"Test: {len(splits.get('test', []))} videos")

In [ ]:
# Find videos with both annotations and features
available_features = set()
for f in os.listdir(EGOVLP_FEATURES_DIR):
    if f.endswith('.npz'):
        # Extract recording_id: "9_8_360p_224.mp4_1s_1s.npz" -> "9_8"
        recording_id = '_'.join(f.split('_')[:2])
        available_features.add(recording_id)

annotated_ids = set(annotations.keys())
common_ids = annotated_ids.intersection(available_features)

print(f"\nVideos with both annotations and features: {len(common_ids)}")
print(f"Sample IDs: {list(common_ids)[:5]}")

## 4. Method 1: Zero-shot Clustering-based Step Localization

Use temporal clustering on EgoVLP features to detect step boundaries.

In [ ]:
import torch
from sklearn.cluster import KMeans
from scipy.signal import find_peaks
import numpy as np

def detect_boundaries_clustering(features, n_clusters='auto', min_cluster_size=5):
    """
    Detect step boundaries using temporal clustering.
    
    Args:
        features: (T, D) array of frame-level features
        n_clusters: number of clusters (steps), 'auto' to estimate
        min_cluster_size: minimum frames per cluster
    
    Returns:
        boundaries: list of (start, end) frame indices
    """
    T, D = features.shape
    
    # Auto-estimate number of clusters if needed
    if n_clusters == 'auto':
        # Heuristic: ~30 seconds per step at 1 FPS
        n_clusters = max(2, min(T // 30, 15))  # 2-15 steps
    
    # K-means clustering
    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
    labels = kmeans.fit_predict(features)
    
    # Find boundary points (where label changes)
    boundaries = []
    current_start = 0
    current_label = labels[0]
    
    for i in range(1, len(labels)):
        if labels[i] != current_label:
            # Check minimum size
            if i - current_start >= min_cluster_size:
                boundaries.append((current_start, i - 1))
                current_start = i
                current_label = labels[i]
    
    # Add last segment
    if T - current_start >= min_cluster_size:
        boundaries.append((current_start, T - 1))
    
    return boundaries

def detect_boundaries_similarity(features, threshold=0.5, min_segment_length=5):
    """
    Alternative: Detect boundaries using cosine similarity changes.
    """
    T, D = features.shape
    
    # Compute cosine similarity between consecutive frames
    features_norm = features / (np.linalg.norm(features, axis=1, keepdims=True) + 1e-8)
    similarities = np.sum(features_norm[:-1] * features_norm[1:], axis=1)
    
    # Find dips in similarity (potential boundaries)
    dissimilarity = 1 - similarities
    peaks, _ = find_peaks(dissimilarity, height=threshold, distance=min_segment_length)
    
    # Convert peaks to boundaries
    boundaries = []
    start = 0
    for peak in peaks:
        boundaries.append((start, peak))
        start = peak + 1
    boundaries.append((start, T - 1))
    
    return boundaries

print("✅ Boundary detection functions defined")

### 4.1 Test on One Video

In [ ]:
# Test on one video
test_id = list(common_ids)[0]
print(f"Testing on: {test_id}")

# Load features
feature_files = [f for f in os.listdir(EGOVLP_FEATURES_DIR) 
                 if f.startswith(test_id.replace('_', '_')) and f.endswith('.npz')]
if feature_files:
    feature_path = os.path.join(EGOVLP_FEATURES_DIR, feature_files[0])
    data = np.load(feature_path)
    features = data['arr_0']  # Shape: (T, 256)
    
    print(f"Features shape: {features.shape}")
    print(f"Duration: {features.shape[0]} seconds (at 1 FPS)")
    
    # Get ground truth
    gt_steps = annotations[test_id].get('steps', [])
    print(f"\nGround truth: {len(gt_steps)} steps")
    for i, step in enumerate(gt_steps[:5]):
        print(f"  [{i+1}] {step.get('start_time', 0):.1f}s - {step.get('end_time', 0):.1f}s: {step.get('description', '')[:50]}")
    
    # Predict boundaries
    pred_boundaries = detect_boundaries_clustering(features, n_clusters='auto')
    print(f"\nPredicted: {len(pred_boundaries)} steps")
    for i, (start, end) in enumerate(pred_boundaries[:5]):
        print(f"  [{i+1}] {start}s - {end}s ({end-start+1} frames)")

### 4.2 Visualize Predictions vs Ground Truth

In [ ]:
import matplotlib.pyplot as plt

def visualize_boundaries(features, gt_boundaries, pred_boundaries, title=""):
    """
    Visualize feature timeline with GT and predicted boundaries.
    """
    fig, axes = plt.subplots(2, 1, figsize=(15, 6), sharex=True)
    
    T = features.shape[0]
    time = np.arange(T)
    
    # Plot feature magnitude
    feature_norm = np.linalg.norm(features, axis=1)
    axes[0].plot(time, feature_norm, alpha=0.6, label='Feature magnitude')
    axes[0].set_ylabel('Feature norm')
    axes[0].set_title(f'{title} - Feature timeline')
    axes[0].grid(True, alpha=0.3)
    
    # Mark GT boundaries
    for start, end in gt_boundaries:
        axes[0].axvspan(start, end, alpha=0.2, color='green', label='GT step' if start == gt_boundaries[0][0] else '')
        axes[0].axvline(start, color='green', linestyle='--', alpha=0.5)
    
    # Mark predicted boundaries
    for start, end in pred_boundaries:
        axes[1].axvspan(start, end, alpha=0.2, color='blue', label='Predicted step' if start == pred_boundaries[0][0] else '')
        axes[1].axvline(start, color='blue', linestyle='--', alpha=0.5)
    
    axes[1].plot(time, feature_norm, alpha=0.6)
    axes[1].set_xlabel('Time (seconds)')
    axes[1].set_ylabel('Feature norm')
    axes[1].set_title('Predicted boundaries')
    axes[1].grid(True, alpha=0.3)
    
    axes[0].legend()
    axes[1].legend()
    plt.tight_layout()
    plt.show()

# Convert GT to frame indices (assuming 1 FPS)
gt_frame_boundaries = [(int(step.get('start_time', 0)), int(step.get('end_time', 0))) 
                       for step in gt_steps]

visualize_boundaries(features, gt_frame_boundaries, pred_boundaries, title=test_id)

## 5. Process All Videos & Extract Step Embeddings

In [ ]:
from tqdm import tqdm

def process_all_videos(video_ids, method='clustering'):
    """
    Process all videos to get step boundaries and embeddings.
    
    Returns:
        results: dict with keys:
            - 'boundaries': {recording_id: [(start, end), ...]}
            - 'step_embeddings': {recording_id: (num_steps, 256)}
            - 'video_labels': {recording_id: 0/1}
    """
    results = {
        'boundaries': {},
        'step_embeddings': {},
        'video_labels': {},
        'activity_names': {},
    }
    
    for recording_id in tqdm(video_ids, desc="Processing videos"):
        try:
            # Find feature file
            feature_files = [f for f in os.listdir(EGOVLP_FEATURES_DIR) 
                           if f.startswith(recording_id.replace('_', '_')) and f.endswith('.npz')]
            
            if not feature_files:
                continue
            
            # Load features
            feature_path = os.path.join(EGOVLP_FEATURES_DIR, feature_files[0])
            data = np.load(feature_path)
            features = data['arr_0']  # (T, 256)
            
            # Detect boundaries
            if method == 'clustering':
                boundaries = detect_boundaries_clustering(features)
            else:
                boundaries = detect_boundaries_similarity(features)
            
            # Extract step embeddings (average features in each segment)
            step_embeddings = []
            for start, end in boundaries:
                step_feat = features[start:end+1].mean(axis=0)  # (256,)
                step_embeddings.append(step_feat)
            
            step_embeddings = np.stack(step_embeddings, axis=0)  # (num_steps, 256)
            
            # Get video label (has errors or not)
            anno = annotations[recording_id]
            has_errors = any(step.get('has_errors', False) for step in anno.get('steps', []))
            video_label = 1 if has_errors else 0
            
            # Store results
            results['boundaries'][recording_id] = boundaries
            results['step_embeddings'][recording_id] = step_embeddings
            results['video_labels'][recording_id] = video_label
            results['activity_names'][recording_id] = anno.get('activity_name', 'unknown')
            
        except Exception as e:
            print(f"Error processing {recording_id}: {e}")
            continue
    
    return results

# Process all available videos
print(f"Processing {len(common_ids)} videos...")
all_results = process_all_videos(list(common_ids), method='clustering')

print(f"\n✅ Processed {len(all_results['boundaries'])} videos successfully")

## 6. Statistics & Analysis

In [ ]:
# Statistics
num_steps_list = [len(bounds) for bounds in all_results['boundaries'].values()]
labels = list(all_results['video_labels'].values())

print("=== Substep 1 Results ===")
print(f"Total videos processed: {len(all_results['boundaries'])}")
print(f"\nVideo labels distribution:")
print(f"  - Correct executions (label=0): {labels.count(0)} ({labels.count(0)/len(labels)*100:.1f}%)")
print(f"  - Has errors (label=1): {labels.count(1)} ({labels.count(1)/len(labels)*100:.1f}%)")
print(f"\nStep detection statistics:")
print(f"  - Avg steps per video: {np.mean(num_steps_list):.1f}")
print(f"  - Min/Max steps: {min(num_steps_list)} / {max(num_steps_list)}")
print(f"  - Median steps: {np.median(num_steps_list):.0f}")

In [ ]:
# Visualize distribution
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Steps distribution
axes[0].hist(num_steps_list, bins=20, edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Number of steps')
axes[0].set_ylabel('Number of videos')
axes[0].set_title('Distribution of Steps per Video')
axes[0].grid(True, alpha=0.3)

# Label distribution
label_counts = [labels.count(0), labels.count(1)]
axes[1].bar(['Correct (0)', 'Has Errors (1)'], label_counts, color=['green', 'red'], alpha=0.7)
axes[1].set_ylabel('Number of videos')
axes[1].set_title('Video Label Distribution')
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## 7. Prepare Data for Teammates (Substeps 2, 3, 4)

Format the output data for downstream tasks.

In [ ]:
# Prepare padded embeddings for Substep 2 (Transformer classifier)
max_steps = max(len(bounds) for bounds in all_results['boundaries'].values())
print(f"Max steps in dataset: {max_steps}")

# Create padded arrays
recording_ids = list(all_results['step_embeddings'].keys())
n_videos = len(recording_ids)

padded_embeddings = np.zeros((n_videos, max_steps, 256), dtype=np.float32)
masks = np.zeros((n_videos, max_steps), dtype=bool)
video_labels = np.zeros(n_videos, dtype=np.int64)

for i, rec_id in enumerate(recording_ids):
    embeddings = all_results['step_embeddings'][rec_id]  # (num_steps, 256)
    num_steps = len(embeddings)
    
    # Fill in embeddings and mask
    padded_embeddings[i, :num_steps] = embeddings
    masks[i, :num_steps] = True
    video_labels[i] = all_results['video_labels'][rec_id]

print(f"\n=== Data for Substep 2 (Task Verification) ===")
print(f"Embeddings shape: {padded_embeddings.shape}  # (N, max_steps, 256)")
print(f"Masks shape: {masks.shape}  # (N, max_steps)")
print(f"Labels shape: {video_labels.shape}  # (N,)")
print(f"\nThis can be fed to a Transformer classifier for video-level prediction.")

In [ ]:
# Save for Substep 2
substep2_output = os.path.join(OUTPUT_DIR, "substep2_data.npz")
np.savez(
    substep2_output,
    embeddings=padded_embeddings,
    masks=masks,
    labels=video_labels,
    recording_ids=np.array(recording_ids, dtype=object),
    max_steps=max_steps
)
print(f"✅ Saved Substep 2 data: {substep2_output}")
print(f"   Size: {os.path.getsize(substep2_output) / (1024*1024):.2f} MB")

In [ ]:
# Save boundaries for Substeps 3 & 4 (Task Graph matching)
substep3_output = os.path.join(OUTPUT_DIR, "step_boundaries.json")

# Convert numpy arrays to lists for JSON serialization
boundaries_json = {
    rec_id: {
        'boundaries': [(int(s), int(e)) for s, e in bounds],
        'num_steps': len(bounds),
        'video_label': int(all_results['video_labels'][rec_id]),
        'activity': all_results['activity_names'][rec_id]
    }
    for rec_id, bounds in all_results['boundaries'].items()
}

with open(substep3_output, 'w') as f:
    json.dump(boundaries_json, f, indent=2)

print(f"✅ Saved step boundaries: {substep3_output}")
print(f"   This can be used for task graph matching in Substeps 3 & 4")

In [ ]:
# Save raw step embeddings (for Substep 3)
substep3_embeddings = os.path.join(OUTPUT_DIR, "step_embeddings.npz")

# Save as dict of arrays
np.savez(
    substep3_embeddings,
    **{rec_id: emb for rec_id, emb in all_results['step_embeddings'].items()}
)

print(f"✅ Saved step embeddings: {substep3_embeddings}")
print(f"   Shape per video: (num_steps, 256)")
print(f"   Use for matching with task graph node embeddings")

## 8. Summary & Deliverables

### ✅ What you've completed:

1. **Step Localization**: Detected step boundaries using zero-shot clustering on EgoVLP features
2. **Step Embeddings**: Extracted step-level embeddings by averaging features in each segment
3. **Data Preparation**: Prepared outputs in the format needed by teammates

### 📦 Deliverables for teammates:

**For Substep 2 (Task Verification baseline)**:
- `substep2_data.npz`:
  - `embeddings`: (N, max_steps, 256) - padded step embeddings
  - `masks`: (N, max_steps) - valid step positions
  - `labels`: (N,) - video-level correctness labels
  - Use this to train a Transformer classifier

**For Substep 3 (Task Graph encoding)**:
- `step_boundaries.json`: Step boundaries for each video
- `step_embeddings.npz`: Variable-length step embeddings per video
- Use these to match detected steps with task graph nodes

**For Substep 4 (GNN classifier)**:
- Same files as Substep 3
- After matching, construct task graph with updated node features
- Feed to GNN for classification

### 🔄 Next steps:

1. Share the output files with teammates
2. Optionally: Compare with ActionFormer predictions (if available)
3. Optionally: Tune clustering parameters for better boundary detection

In [ ]:
# Final summary
print("="*60)
print("EXTENSION SUBSTEP 1 COMPLETE!")
print("="*60)
print(f"\n📊 Processing Summary:")
print(f"   - Videos processed: {len(all_results['boundaries'])}")
print(f"   - Total steps detected: {sum(num_steps_list)}")
print(f"   - Avg steps per video: {np.mean(num_steps_list):.1f}")
print(f"\n📁 Output files in: {OUTPUT_DIR}")
for fname in ['substep2_data.npz', 'step_boundaries.json', 'step_embeddings.npz']:
    fpath = os.path.join(OUTPUT_DIR, fname)
    if os.path.exists(fpath):
        size_mb = os.path.getsize(fpath) / (1024*1024)
        print(f"   ✅ {fname} ({size_mb:.2f} MB)")

print(f"\n🎯 Ready for teammates to implement Substeps 2, 3, 4!")

## Optional: Evaluate Against Ground Truth

In [ ]:
# Optional: Compute IoU/F1 metrics against ground truth boundaries
def compute_iou(pred_bounds, gt_bounds, video_length):
    """
    Compute IoU between predicted and ground truth boundaries.
    """
    pred_mask = np.zeros(video_length, dtype=bool)
    gt_mask = np.zeros(video_length, dtype=bool)
    
    for start, end in pred_bounds:
        pred_mask[start:end+1] = True
    
    for start, end in gt_bounds:
        gt_mask[start:end+1] = True
    
    intersection = np.logical_and(pred_mask, gt_mask).sum()
    union = np.logical_or(pred_mask, gt_mask).sum()
    
    return intersection / union if union > 0 else 0.0

# Compute average IoU
ious = []
for rec_id in list(all_results['boundaries'].keys())[:10]:  # Sample
    pred_bounds = all_results['boundaries'][rec_id]
    
    gt_steps = annotations[rec_id].get('steps', [])
    gt_bounds = [(int(s.get('start_time', 0)), int(s.get('end_time', 0))) for s in gt_steps]
    
    # Get video length from features
    feature_files = [f for f in os.listdir(EGOVLP_FEATURES_DIR) 
                    if f.startswith(rec_id.replace('_', '_')) and f.endswith('.npz')]
    if feature_files:
        data = np.load(os.path.join(EGOVLP_FEATURES_DIR, feature_files[0]))
        video_length = data['arr_0'].shape[0]
        
        iou = compute_iou(pred_bounds, gt_bounds, video_length)
        ious.append(iou)

if ious:
    print(f"\nAverage IoU (sample): {np.mean(ious):.3f}")
    print("Note: This is just for reference. The important output is the step embeddings.")